In [304]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

# set env
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)


# seq_dg_230703.txt 是在 0620文件的基础上新增了两列信息；
# dG_ori 列表示原始的dG信息；
# complex_type 列表示复合物类型,wt表示分类为野生型,mut表示分类为突变体,dele则是需要删除的复合物;
# dist_index 列用于索引 dist文件,dist文件在mmseqs文件夹下,共有5个文件,蛋白和四种类型的核酸,
# fident表示距离,范围在0-1之间; 
# 相同的组合,例如 protein_6 和 protein_12之间的dist是相同的,
# 若在核酸的输出文件中检索不到两两之间的相似性，请用0填充,说明其相似性很低;

def convertCsv(df):
    if 'key_nucleic_acids' not in df.columns:
        df.insert(len(df.columns), 'key_nucleic_acids', 'None', allow_duplicates=False)
    for i in range(len(df)):
        row = df.loc[i]
        na_type = row['nucleic_acid_type_new']
        if na_type == 'dsDNA':
            df.loc[i, 'key_nucleic_acids'] = row['newnafea_na_job'].replace('new_DNA', 'Double_DNA')
        elif na_type == 'dsRNA':
            df.loc[i, 'key_nucleic_acids'] = row['newnafea_na_job'].replace('new_RNA', 'Double_RNA')
        elif na_type == 'ssDNA':
            df.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssDNA', 'Single_DNA_')
        elif na_type == 'ssRNA':
            df.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssRNA', 'Single_RNA_')
    return df


data_root = '../data/_datasets/cluster_res/'

new_ma_f = 'seq_dg_230703.txt'
new_ma_f = os.path.join(data_root, new_ma_f)
new_madf = pd.read_csv(new_ma_f, sep='\t', low_memory=False)
# print(len(new_madf))
# new_madf.head(3)

# if 'key_nucleic_acids' not in new_madf.columns:
#     new_madf.insert(len(new_madf.columns), 'key_nucleic_acids', 'None', allow_duplicates=False)

# for i in range(len(new_madf)):
#     row = new_madf.loc[i]
#     na_type = row['nucleic_acid_type_new']
#     if na_type == 'dsDNA' or na_type == 'dsRNA':
#         double_s_key = row['newnafea_na_job']
#         new_madf.loc[i, 'key_nucleic_acids'] = double_s_key.replace('new', 'Double')
#     elif na_type == 'ssDNA':
#         new_madf.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssDNA', 'Single_DNA_')
#     elif na_type == 'ssRNA':
#         new_madf.loc[i, 'key_nucleic_acids'] = row['na_index'].replace('ssRNA', 'Single_RNA_')
new_madf = convertCsv(new_madf)
new_madf.to_csv(os.path.join(data_root, 'seq_dg_v01.txt'), sep='\t', index=False)


wtf = 'wt.tsv'
wtf = os.path.join(data_root, wtf)
wtdf = pd.read_csv(wtf, sep='\t', low_memory=False)
wtdf = convertCsv(wtdf)
wtdf.to_csv(os.path.join(data_root, 'wt_v01.tsv'), sep='\t', index=False)
# The dataset "seq_dg_230702.txt" contains information about dG. 
# The "nucleic_acid_type_new" column represents the four types of nucleic acids. 
# The "dG_ori" column contains the original dG values obtained from the literature. 
# The "complex_type" column indicates whether the complex is of the wild type or mutant type, 
# allowing us to divide the dataset based on this information. 
# By the way,If the value in the "complex_type" column is "dele," 
# it indicates that the complex needs to be removed from the dataset.

# f = os.path.join(data_root, 'seq_dg_230620.txt')
# old_madf = pd.read_csv(f, sep='\t', low_memory=False)
# print(len(old_madf))
# old_madf.head(3)

#230703 
#双链核酸的相似性文件已经修改,是格式的问题,现在能够直接按照 na_index 列进行索引;
#下面是例子;
#dsdnaSim = pd.read_table('/data/sswwkk/Protein_sequence/quanzhe_share/absolute_affinity/date/230702/sim_matrix/dsDNA_ident_matrix.txt',index_col=0)
#seq_dg = pd.read_csv('/data/sswwkk/Protein_sequence/quanzhe_share/absolute_affinity/date/230702/seq_dg_230702.txt',sep='\t')
#seq_dg_dsDNA = seq_dg[seq_dg['nucleic_acid_type_new']=='dsDNA']
#通过以下方式索引
#dsdnaSim.loc['dsdnanew_DNA_3312','dsdnanew_DNA_2549']

# newnafea_na_job：用于索引 2个双链cluster文件 和 2个双链相似度文件
# na_index： 用于索引 2个单链cluster文件 和 2个单链相似度文件
# nucleic_acid_type_new: 核算的种类

def read2df(root, f):
    f = os.path.join(root, f)
    return pd.read_csv(f, sep='\t') # , low_memory=False

def write2csv(root, f, df):
    f = os.path.join(root, f)
    df.to_csv(f, sep='\t', index=False)
    print("write to {}".format(f))
    
pwtc_f = 'protein_wt_cluster.tsv'
ddna_f = 'dsDNA_80_cluster.txt'
drna_f = 'dsRNA_80_cluster.txt'
sdna_f = 'ssDNA_80_cluster.txt'
srna_f = 'ssRNA_80_cluster.txt'

pwtc_df = read2df(data_root, pwtc_f)
ddna_df = read2df(data_root, ddna_f)
drna_df = read2df(data_root, drna_f)
sdna_df = read2df(data_root, sdna_f)
srna_df = read2df(data_root, srna_f)

def convertD(df, cols):
    return df.apply(lambda x: x.replace(
                {'new_DNA': 'Double_DNA', 
                 'new_RNA': 'Double_RNA',
                 'ssDNA': 'Single_DNA_',
                 'ssRNA': 'Single_RNA_'
                },
                regex=True))

changed_cols = ['protein_index', 'cluster_member']

ddna_df = convertD(ddna_df, changed_cols)
drna_df = convertD(drna_df, changed_cols)
sdna_df = convertD(sdna_df, changed_cols)
srna_df = convertD(srna_df, changed_cols)


write2csv(data_root, ddna_f, ddna_df)
write2csv(data_root, drna_f, drna_df)
write2csv(data_root, sdna_f, sdna_df)
write2csv(data_root, srna_f, srna_df)



dir_sim = 'similarity'

sim_ddna_f = 'dsdna_dist_230703.txt'  
sim_drna_f = 'dsrna_dist_230703.txt'
sim_sdna_f = 'ssdna_dist_230703.txt'  
sim_srna_f = 'ssrna_dist_230703.txt'

# sim_ddna_df = pd.read_table(os.path.join(data_root, dir_sim, sim_ddna_f), index_col=0)
# sim_drna_df = pd.read_table(os.path.join(data_root, dir_sim, sim_drna_f), index_col=0)
# sim_sdna_df = pd.read_table(os.path.join(data_root, dir_sim, sim_sdna_f), index_col=0)
# sim_srna_df = pd.read_table(os.path.join(data_root, dir_sim, sim_srna_f), index_col=0)
sim_dir = os.path.join(data_root, dir_sim)

sim_ddna_df = read2df(sim_dir, sim_ddna_f)
sim_drna_df = read2df(sim_dir, sim_drna_f)
sim_sdna_df = read2df(sim_dir, sim_sdna_f)
sim_srna_df = read2df(sim_dir, sim_srna_f)

changed_cols = ['query', 'target']
sim_ddna_df = convertD(sim_ddna_df, changed_cols)
sim_drna_df = convertD(sim_drna_df, changed_cols)
sim_sdna_df = convertD(sim_sdna_df, changed_cols)
sim_srna_df = convertD(sim_srna_df, changed_cols)

write2csv(sim_dir, sim_ddna_f, sim_ddna_df)
write2csv(sim_dir, sim_drna_f, sim_drna_df)
write2csv(sim_dir, sim_sdna_f, sim_sdna_df)
write2csv(sim_dir, sim_srna_f, sim_srna_df)

print('done')

write to ../data/_datasets/cluster_res/dsDNA_80_cluster.txt
write to ../data/_datasets/cluster_res/dsRNA_80_cluster.txt
write to ../data/_datasets/cluster_res/ssDNA_80_cluster.txt
write to ../data/_datasets/cluster_res/ssRNA_80_cluster.txt
write to ../data/_datasets/cluster_res/similarity/dsdna_dist_230703.txt
write to ../data/_datasets/cluster_res/similarity/dsrna_dist_230703.txt
write to ../data/_datasets/cluster_res/similarity/ssdna_dist_230703.txt
write to ../data/_datasets/cluster_res/similarity/ssrna_dist_230703.txt
done


In [377]:
'type_nucleic_acid' in tmp_df.columns

False

In [378]:
tmp_df = pd.read_csv(os.path.join(data_root, 'seq_dg_v01_1.txt'), sep='\t', low_memory=False)
tmp_df.head(5)


,complex_id,dataset_id,UniProt,protein_sequence,nucleotide_sequence,nucleic_acid_mutation,protein_mutation,nucleic_acid_type,nucleic_acid_type_new,Kd,dG,pH,Tm,Method,pmid,data_source,protein_na_pair,multiple_dg,max_min,max_min_judge,dG_ori,pdb_id,len_p1,len_p2,protein_chain,na_chain,identity_protein_seq,identity_na_seq,UniProt_type,protein_sequence_len,wt_af2_file_download,mo_pdb_chain,bs_score,nucleotide_sequence_one,nucleotide_sequence_two,nucleotide_sequence_two_reverse,align_score,len_nucleotide_sequence_one,len_nucleotide_sequence_two,coverage_nucleotide_sequence_one,coverage_nucleotide_sequence_two,coverage_mean_nucleotide_sequence,protein_index,cluster_protein_50_index,cluster_protein_50_number,cluster_protein_50_member,na_index,cluster_na_50_index,cluster_na_50_number,cluster_na_50_member,protein_na_pair_index,protein_50_na_50_member,protein_50_na_50_number,cluster_protein_na_pair_index,dataset_id_pdb_id_protein_index,protein_species,na_jobid,nucleotide_sequence_fornew_na_fea,newnafea_na_job,AF2_filname,AF2_protein_sequence_len,pLDDT_median,protein_50_na_50_member_Li_clustertype,protein_50_na_50_number_Li_clustertype,cluster_protein_na_pair_index_Li_clustertype,mutation_type,protein_sequence_number,protein_sequence_wild,complex_type,dist_index,key_nucleic_acids,nuclass,wt_complex,pclass,base_class,avg_dist_to_others
0,0,pnat_143,P04150,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,CGCCTCCGGGAGAGCT,NONE,WILD,ssDNA,ssDNA,3.6e-07,-8.79,7.5,298.0,Isothermal titration calorimetry,23222642,pnat,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,1,0.0,good,-8.79,3fyl,"A:74,B:76","C:16,D:16",A_1.B_1,C_1.D_1,0.1,0.25,Swiss-Prot,777,yes,"1m2z_A,6cfn_C","0.7293488372093023,0.23546511627906977",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_6,Cluster_43,9,"protein_6,protein_965,protein_966,protein_1059...",ssDNA0,Cluster_1008,1,ssDNA0,protein_6_ssDNA0,protein_6_ssDNA0,1,cluster_pair_0,pnat_143_3fyl_protein_6,Rattus norvegicus,DNA_0022,CGCCTCCGGGAGAGCT,new_DNA_0,AF-P04150-F1-model_v4.pdb,777,40.51,"protein_6_ssDNA0,protein_34_ssDNA28,protein_34...",29,cluster_pair_0,WILD,6,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,wt,ssDNA0,Single_DNA_0,SDNACluster_1394,True,PCluster_445,SDNACluster_1394-PCluster_445,3.409316
1,1,pnat_13395,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCTTCCCT,NONE,WILD,DNA,ssDNA,1.83e-06,-7.83,7.8,298.0,Fluorescence,18729380,pnat,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,1,0.0,good,-7.83,1jmc,A:238,B:8,A_1,B_1,0.39,0.12,Swiss-Prot,616,yes,"1fgu_A,1l1o_C,2b29_A","0.4446617647058823,0.3088235294117647,0.214705...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_12,Cluster_30,22,"protein_12,protein_2647,protein_2658,protein_2...",ssDNA1,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA1,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13395_1jmc_protein_12,Homo sapiens,DNA_0225,CTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCTTCCCT,new_DNA_1,AF-P27694-F1-model_v4.pdb,616,92.17,"protein_12_ssDNA1,protein_12_ssDNA2,protein_12...",47,cluster_pair_1,WILD,22,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,wt,ssDNA1,Single_DNA_1,SDNACluster_1441,True,PCluster_446,SDNACluster_1441-PCluster_446,3.373016
2,2,pnat_13396,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CCCTTCTTTCTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCT...,NONE,WILD,DNA,ssDNA,1.39e-06,-7.99,7.8,298.0,Fluorescence,18729380,pnat,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,1,0.0,good,-7.99,1jmc,A:238,B:8,A_1,B_1,0.39,0.12,Swiss-Prot,616,yes,"1fgu_A,1l1o_C,2b29_A","0.4446617647058823,0.3088235294117647,0.214705...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_12,Cluster_30,22,"protein_12,protein_2647,protein_2658,protein_2...",ssDNA2,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA2,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13396_1jmc_protein_12,Homo sapiens,DNA_0226,CCCTTCTTTCTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCT...,new_DNA_2,AF-P27694-F1-model_v4.pdb,616,92

In [368]:
# a = np.arange(10).reshape(1,-1)
# mat = np.concatenate([a,a,a,a,a,a,a,a,a], axis=0)
# i = np.array([3,5,7])
# j = np.array([1,2,6,10,11])
# mat[i, j]

# x = np.array([1,2,6,10,11]).reshape(-1, 1)
# x = np.repeat(x, 3, axis=1)
# print(x)
# for i in range(3):
#     x[:,i] = x[:,i] * (i+1)
# print(x)
# x = np.repeat(x, 3, axis=1)
# print(x)

x = np.array([1,2,6,10,11]).reshape(1, -1)
x = np.repeat(x, 3, axis=0)
print(x)
for i in range(3):
    x[i,:] = x[i,:] * (i+1)
print(x)
x = np.repeat(x, 3, axis=0)
print(x)

[[ 1  2  6 10 11]
 [ 1  2  6 10 11]
 [ 1  2  6 10 11]]
[[ 1  2  6 10 11]
 [ 2  4 12 20 22]
 [ 3  6 18 30 33]]
[[ 1  2  6 10 11]
 [ 1  2  6 10 11]
 [ 1  2  6 10 11]
 [ 2  4 12 20 22]
 [ 2  4 12 20 22]
 [ 2  4 12 20 22]
 [ 3  6 18 30 33]
 [ 3  6 18 30 33]
 [ 3  6 18 30 33]]


In [480]:
def addColumnByName(df, col_name, set_front=False, set_str=True):
  pos = 1 if set_front else len(df.columns)
  fill_val = 'None' if set_str else -1
  if col_name not in df.columns:
    df.insert( pos, col_name, fill_val, allow_duplicates=False )
    print('add a new column {} that does not exist'.format(col_name))
  else:
    print('column {} already exists'.format(col_name))
    
def updateColumnNaType(df):
  col_name = 'key_nucleic_acids'
  addColumnByName(df, col_name)
  # for i in range(len(df)):
  #   row = df.loc[i]
  #   na_type = row['nucleic_acid_type_new']
  #   if na_type == 'dsDNA':
  #       df.loc[i, col_name] = row['newnafea_na_job'].replace('new_DNA', 'Double_DNA')
  #   elif na_type == 'dsRNA':
  #       df.loc[i, col_name] = row['newnafea_na_job'].replace('new_RNA', 'Double_RNA')
  #   elif na_type == 'ssDNA':
  #       df.loc[i, col_name] = row['na_index'].replace('ssDNA', 'Single_DNA_')
  #   elif na_type == 'ssRNA':
  #       df.loc[i, col_name] = row['na_index'].replace('ssRNA', 'Single_RNA_')
  replace_values = {'new_DNA' : 'Double_DNA', 'new_RNA' : 'Double_RNA'}
  sst = df['nucleic_acid_type_new'] == 'dsDNA'
  ss = df[sst]['newnafea_na_job'].str.replace(replace_values)
  df.loc[ss.index, 'key_nucleic_acids'] = ss

  sst = df['nucleic_acid_type_new'] == 'dsRNA'
  ss = df[sst]['newnafea_na_job'].str.replace('new_RNA', 'Double_RNA')
  df.loc[ss.index, 'key_nucleic_acids'] = ss

  sst = df['nucleic_acid_type_new'] == 'ssDNA'
  ss = df[sst]['na_index'].str.replace('ssDNA', 'Single_DNA_')
  df.loc[ss.index, 'key_nucleic_acids'] = ss

  sst = df['nucleic_acid_type_new'] == 'ssRNA'
  ss = df[sst]['na_index'].str.replace('ssRNA', 'Single_RNA_')
  df.loc[ss.index, 'key_nucleic_acids'] = ss

  return df

def updateColumnComplex(df):
  col_name = 'key_complex'
  addColumnByName(df, col_name, set_front=True)

  return df

In [499]:
data_root = '../data/_datasets/cluster_res/'
main_f = 'seq_dg_v01_1.txt'


main_f = os.path.join(data_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

all_cls_set1 = set(main_df['base_class'].values)
all_cls_set1

{'DDNACluster_359-PCluster_628',
 'SDNACluster_491-PCluster_474',
 'SDNACluster_484-PCluster_444',
 'DDNACluster_123-PCluster_481',
 'SDNACluster_1334-PCluster_356',
 'DDNACluster_527-PCluster_192',
 'DDNACluster_491-PCluster_694',
 'SRNACluster_676-PCluster_81',
 'SDNACluster_1216-PCluster_436',
 'SRNACluster_693-PCluster_348',
 'SRNACluster_82-PCluster_322',
 'SRNACluster_659-PCluster_64',
 'DDNACluster_1073-PCluster_433',
 'SDNACluster_275-PCluster_121',
 'SDNACluster_1440-PCluster_446',
 'SDNACluster_1138-PCluster_531',
 'SRNACluster_464-PCluster_49',
 'DDNACluster_839-PCluster_478',
 'SDNACluster_278-PCluster_400',
 'DDNACluster_410-PCluster_534',
 'SRNACluster_18-PCluster_65',
 'SRNACluster_642-PCluster_336',
 'SDNACluster_1212-PCluster_413',
 'SDNACluster_870-PCluster_122',
 'DDNACluster_763-PCluster_279',
 'DDNACluster_299-PCluster_634',
 'SDNACluster_1286-PCluster_441',
 'SDNACluster_783-PCluster_420',
 'SDNACluster_494-PCluster_610',
 'SDNACluster_326-PCluster_400',
 'SRNAClu

In [484]:
data_root = '../data/_datasets/cluster_res/'
### from_files located in orig_root
orig_root = 'original_files'
### from files
main_f = 'seq_dg_230703.txt'
wt_f = 'wt.tsv'
### save to files
new_main_f = 'seq_dg_v02.txt'
new_wt_f = 'wt_v02.tsv'

main_f = os.path.join(data_root, orig_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

wt_f = os.path.join(data_root, orig_root, wt_f)
wt_df = pd.read_csv(wt_f, sep='\t', low_memory=False)

def updateColumnComplex(df):
  col_name = 'key_complex'
  addColumnByName(df, col_name, set_front=True, set_str=False)
  
  curr_id = 0
  for k, subdf in df.groupby(['protein_sequence', 'nucleotide_sequence', 'nucleic_acid_type_new']):
    ss = subdf['exp_id']
    df.loc[ss.index, col_name] = int( ss.values.min() )
    curr_id += 1
  print("totally have {} unique complex".format(curr_id))



main_df.rename(columns={'complex_id': 'exp_id'}, inplace=True)
updateColumnComplex(main_df)
main_df

add a new column key_complex that does not exist
totally have 9062 unique complex


,exp_id,key_complex,dataset_id,UniProt,protein_sequence,nucleotide_sequence,nucleic_acid_mutation,protein_mutation,nucleic_acid_type,nucleic_acid_type_new,Kd,dG,pH,Tm,Method,pmid,data_source,protein_na_pair,multiple_dg,max_min,max_min_judge,dG_ori,pdb_id,len_p1,len_p2,protein_chain,na_chain,identity_protein_seq,identity_na_seq,UniProt_type,protein_sequence_len,wt_af2_file_download,mo_pdb_chain,bs_score,nucleotide_sequence_one,nucleotide_sequence_two,nucleotide_sequence_two_reverse,align_score,len_nucleotide_sequence_one,len_nucleotide_sequence_two,coverage_nucleotide_sequence_one,coverage_nucleotide_sequence_two,coverage_mean_nucleotide_sequence,protein_index,cluster_protein_50_index,cluster_protein_50_number,cluster_protein_50_member,na_index,cluster_na_50_index,cluster_na_50_number,cluster_na_50_member,protein_na_pair_index,protein_50_na_50_member,protein_50_na_50_number,cluster_protein_na_pair_index,dataset_id_pdb_id_protein_index,protein_species,na_jobid,nucleotide_sequence_fornew_na_fea,newnafea_na_job,AF2_filname,AF2_protein_sequence_len,pLDDT_median,protein_50_na_50_member_Li_clustertype,protein_50_na_50_number_Li_clustertype,cluster_protein_na_pair_index_Li_clustertype,mutation_type,protein_sequence_number,protein_sequence_wild,complex_type,dist_index
0,0,0,pnat_143,P04150,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,CGCCTCCGGGAGAGCT,NONE,WILD,ssDNA,ssDNA,3.6e-07,-8.790,7.5,298.0,Isothermal titration calorimetry,23222642,pnat,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,1,0.00,good,-8.79,3fyl,"A:74,B:76","C:16,D:16",A_1.B_1,C_1.D_1,0.1,0.25,Swiss-Prot,777,yes,"1m2z_A,6cfn_C","0.7293488372093023,0.23546511627906977",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_6,Cluster_43,9,"protein_6,protein_965,protein_966,protein_1059...",ssDNA0,Cluster_1008,1,ssDNA0,protein_6_ssDNA0,protein_6_ssDNA0,1,cluster_pair_0,pnat_143_3fyl_protein_6,Rattus norvegicus,DNA_0022,CGCCTCCGGGAGAGCT,new_DNA_0,AF-P04150-F1-model_v4.pdb,777,40.51,"protein_6_ssDNA0,protein_34_ssDNA28,protein_34...",29,cluster_pair_0,WILD,6,MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSP...,wt,ssDNA0
1,1,1,pnat_13395,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCTTCCCT,NONE,WILD,DNA,ssDNA,1.83e-06,-7.830,7.8,298.0,Fluorescence,18729380,pnat,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,1,0.00,good,-7.83,1jmc,A:238,B:8,A_1,B_1,0.39,0.12,Swiss-Prot,616,yes,"1fgu_A,1l1o_C,2b29_A","0.4446617647058823,0.3088235294117647,0.214705...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_12,Cluster_30,22,"protein_12,protein_2647,protein_2658,protein_2...",ssDNA1,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA1,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13395_1jmc_protein_12,Homo sapiens,DNA_0225,CTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCTTCCCT,new_DNA_1,AF-P27694-F1-model_v4.pdb,616,92.17,"protein_12_ssDNA1,protein_12_ssDNA2,protein_12...",47,cluster_pair_1,WILD,22,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,wt,ssDNA1
2,2,2,pnat_13396,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,CCCTTCTTTCTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCT...,NONE,WILD,DNA,ssDNA,1.39e-06,-7.990,7.8,298.0,Fluorescence,18729380,pnat,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,1,0.00,good,-7.99,1jmc,A:238,B:8,A_1,B_1,0.39,0.12,Swiss-Prot,616,yes,"1fgu_A,1l1o_C,2b29_A","0.4446617647058823,0.3088235294117647,0.214705...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_12,Cluster_30,22,"protein_12,protein_2647,protein_2658,protein_2...",ssDNA2,Cluster_1009,2,"ssDNA1,ssDNA2",protein_12_ssDNA2,"protein_12_ssDNA1,protein_12_ssDNA2",2,cluster_pair_1,pnat_13396_1jmc_protein_12,Homo sapiens,DNA_0226,CCCTTCTTTCTCTTCCCCCTCTCCTTCTTGCGCTCTTCCTTCCCCT...,new_DNA_2,AF-P27694-F1-model_v4.pdb,616,92.17,"protein_12_ssDNA1,protein_12_ssDNA2,protein_12...",47,cluster_pair_1,WILD,22,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,wt,ssDNA2
3,3,3,pnat_14707,Q8RNV8,MKIWSKEEVVNKLHEIKNKGYLSVPTDMFRTDDGVVGQILERQFGV...,CGCACGACTTCC

In [488]:
####### check

data_root = '../data/_datasets/cluster_res/'

main_f = 'seq_dg_v02.txt'
wt_f = 'wt_v02.tsv'

main_f = os.path.join(data_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

wt_f = os.path.join(data_root, wt_f)
wt_df = pd.read_csv(wt_f, sep='\t', low_memory=False)

In [493]:
main_df[main_df['key_complex'] == 218]

,exp_id,key_complex,dataset_id,UniProt,protein_sequence,nucleotide_sequence,nucleic_acid_mutation,protein_mutation,nucleic_acid_type,nucleic_acid_type_new,Kd,dG,pH,Tm,Method,pmid,data_source,protein_na_pair,multiple_dg,max_min,max_min_judge,dG_ori,pdb_id,len_p1,len_p2,protein_chain,na_chain,identity_protein_seq,identity_na_seq,UniProt_type,protein_sequence_len,wt_af2_file_download,mo_pdb_chain,bs_score,nucleotide_sequence_one,nucleotide_sequence_two,nucleotide_sequence_two_reverse,align_score,len_nucleotide_sequence_one,len_nucleotide_sequence_two,coverage_nucleotide_sequence_one,coverage_nucleotide_sequence_two,coverage_mean_nucleotide_sequence,protein_index,cluster_protein_50_index,cluster_protein_50_number,cluster_protein_50_member,na_index,cluster_na_50_index,cluster_na_50_number,cluster_na_50_member,protein_na_pair_index,protein_50_na_50_member,protein_50_na_50_number,cluster_protein_na_pair_index,dataset_id_pdb_id_protein_index,protein_species,na_jobid,nucleotide_sequence_fornew_na_fea,newnafea_na_job,AF2_filname,AF2_protein_sequence_len,pLDDT_median,protein_50_na_50_member_Li_clustertype,protein_50_na_50_number_Li_clustertype,cluster_protein_na_pair_index_Li_clustertype,mutation_type,protein_sequence_number,protein_sequence_wild,complex_type,dist_index,key_nucleic_acids
217,218,218,pnat_16383,P10074,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,TTAGGG,NONE,WILD,DNA,ssDNA,1.7e-07,-9.08,7.5,293.0,Fluorescence,29134956,pnat,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,9,0.0,lower_1,-9.08,5yj3,"C:70,D:66","A:18,B:18",C_1.D_1,A_1.B_1,0.1,0.5,Swiss-Prot,688,yes,"3b84_A,5yj3_C","0.5890989010989012,0.3407362637362638",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_233,Cluster_449,6,"protein_233,protein_1942,protein_1943,protein_...",ssDNA118,Cluster_746,1,ssDNA118,protein_233_ssDNA118,"protein_233_ssDNA118,protein_1942_ssDNA118,pro...",6,cluster_pair_117,pnat_16383_5yj3_protein_233,Homo sapiens,DNA_0355,TTAGGG,new_DNA_118,AF-P10074-F1-model_v4.pdb,688,75.26,"protein_233_ssDNA118,protein_1883_ssDNA118,pro...",14,cluster_pair_92,WILD,6,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,wt,ssDNA118,Single_DNA_118
976,982,218,pronab_16120,P10074,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,TTAGGG,wild,WILD,DNA,ssDNA,1.7e-07,-9.08,7.5,293.0,Fluorescence,29134956,pronab,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,9,0.0,lower_1,-9.08,5yj3,"C:70,D:66","A:18,B:18",C_1.D_1,A_1.B_1,0.1,0.5,Swiss-Prot,688,yes,"3b84_A,5yj3_C","0.5890989010989012,0.3407362637362638",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_233,Cluster_449,6,"protein_233,protein_1942,protein_1943,protein_...",ssDNA118,Cluster_746,1,ssDNA118,protein_233_ssDNA118,"protein_233_ssDNA118,protein_1942_ssDNA118,pro...",6,cluster_pair_117,pronab_16120_5yj3_protein_233,Homo sapiens,DNA_0355,TTAGGG,new_DNA_118,AF-P10074-F1-model_v4.pdb,688,75.26,"protein_233_ssDNA118,protein_1883_ssDNA118,pro...",14,cluster_pair_92,WILD,6,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,wt,ssDNA118,Single_DNA_118
2224,2236,218,pnat_16391,P10074,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,TTAGGG,NONE,WILD,DNA,ssDNA,1.7e-07,-9.08,7.5,293.0,Fluorescence,29134956,pnat,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,9,0.0,lower_1,-9.08,NONE,NONE,NONE,NONE,NONE,NONE,NONE,Swiss-Prot,688,yes,"3b84_A,5yj3_C","0.5890989010989012,0.3407362637362638",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,protein_233,Cluster_449,6,"protein_233,protein_1942,protein_1943,protein_...",ssDNA118,Cluster_746,1,ssDNA118,protein_233_ssDNA118,"protein_233_ssDNA118,protein_1942_ssDNA118,pro...",6,cluster_pair_117,pnat_16391_NONE_protein_233,Homo sapiens,DNA_0355,TTAGGG,new_DNA_118,AF-P10074-F1-model_v4.pdb,688,75.26,"protein_233_ssDNA118,protein_1883_ssDNA118,pro...",14,cluster_pair_92,WILD,6,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,wt,ssDNA118,Single_DNA_118
2225,2237,218,pnat_16392,P10074,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,TTAGGG,NONE,WILD,DNA,ssDNA,1.7e-07,-9.08,

In [494]:
wt_df[wt_df['key_complex'] == 218]

,UniProt,key_complex,protein_index,na_index,mutation_type,protein_mutation,protein_sequence,nucleotide_sequence,nucleic_acid_type_new,dG,newnafea_na_job,na_jobid,protein_na_pair_index,exp_id,key_nucleic_acids
105,P10074,218,protein_233,ssDNA118,WILD,WILD,MDGSFVQHSVRVLQELNKQREKGQYCDATLDVGGLVFKAHWSVLAC...,TTAGGG,ssDNA,-9.08,new_DNA_118,DNA_0355,protein_233_ssDNA118,218,Single_DNA_118


In [495]:
clsPair2dist_f = os.path.join(data_root, 'output_v01.npz')
if os.path.exists(clsPair2dist_f):
  clsPair2dist_npzData = np.load(clsPair2dist_f, allow_pickle=True)
  map_cls_pair2dist = clsPair2dist_npzData['map_cls_pair2dist'].item()
  print('similarity data of cluster pair exists, read: {}'.format(clsPair2dist_f))

similarity data of cluster pair exists, read: ../data/_datasets/cluster_res/output_v01.npz


In [510]:
import sys
np.set_printoptions(threshold=sys.maxsize)
data_root = '../data/_datasets/cluster_res/'
main_f = 'seq_dg_v02_1.txt'

main_f = os.path.join(data_root, main_f)
main_df = pd.read_csv(main_f, sep='\t', low_memory=False)

# ks=np.unique(main_df['base_class'].values)
# len(ks)

4227

In [512]:
other_root = '../data/pri_data/'

train_df = pd.read_csv( os.path.join(other_root, 'train', 'train.csv'), sep='\t', low_memory=False)
test_df = pd.read_csv( os.path.join(other_root, 'test', 'test.csv'), sep='\t', low_memory=False)

In [518]:
uid2all = dict()
for k, subdf in main_df.groupby('key_complex'):
    uid2all[k] = subdf['exp_id'].values

In [521]:
n_uniq = 0
n_same = 0
nxx = 0
nyy = 0
for k, subdf in main_df.groupby('key_complex'):
    m = test_df['complex_id'].isin(subdf['exp_id'].values)
    ss = test_df['complex_id'][m]
    cids = ss.values
    if len(cids) > 0:
        if len(subdf['exp_id'].values) == 1:
            n_uniq += 1
        else:
            n_same += 1
            n = len(subdf['exp_id'].values) - len(cids)
            if n == 0:
                nyy += 1
            nxx += n
n_uniq, n_same, nxx, nyy

(1358, 2479, 5402, 0)